# Questão 10 - Análise de Sentimentos em Redes Sociais
**Autor:** Leonardo Paz  
**Data:** Outubro 2024

In [ ]:
# Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

# Download NLTK data
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

print("Bibliotecas importadas")

In [ ]:
# Criar dados de exemplo (reviews de produtos)
np.random.seed(42)
reviews_positivos = [
    "Adorei este produto, qualidade excelente!",
    "Muito bom, recomendo a todos",
    "Funciona perfeitamente, superou expectativas",
    "Ótimo custo-benefício, vale cada centavo",
    "Produto incrível, entrega rápida",
    "Qualidade superior, muito satisfeito",
    "Excelente desempenho, recomendo",
    "Surpreendentemente bom, adorei",
    "Perfeito para minhas necessidades",
    "Muito eficiente, resultado excelente"
]

reviews_negativos = [
    "Péssima qualidade, não recomendo",
    "Produto veio com defeito, muito ruim",
    "Desapontado, não funciona como esperado",
    "Qualidade inferior, dinheiro jogado fora",
    "Horrível, pior compra que já fiz",
    "Não atendeu minhas expectativas",
    "Problemas desde o primeiro uso",
    "Muito decepcionante, evitem",
    "Funcionamento irregular, muito frustrante",
    "Arrependido da compra, produto ruim"
]

# Criar dataset balanceado
reviews = reviews_positivos + reviews_negativos
sentimentos = [1] * len(reviews_positivos) + [0] * len(reviews_negativos)

df = pd.DataFrame({'review': reviews, 'sentimento': sentimentos})
print("Dataset criado:")
print(f"Shape: {df.shape}")
print(f"Distribuição: {df['sentimento'].value_counts()}")
df.head()

In [ ]:
# Pré-processamento de texto
stop_words = set(stopwords.words('portuguese'))
stemmer = PorterStemmer()

def preprocessar_texto(texto):
    # Converter para minúsculo
    texto = texto.lower()
    # Remover caracteres especiais
    texto = re.sub(r'[^\w\s]', '', texto)
    # Tokenização
    tokens = word_tokenize(texto)
    # Remover stopwords e stemming
    tokens = [stemmer.stem(token) for token in tokens if token not in stop_words and len(token) > 2]
    return ' '.join(tokens)

df['review_limpo'] = df['review'].apply(preprocessar_texto)
print("Texto pré-processado:")
print(df[['review', 'review_limpo']].head())

In [ ]:
# WordCloud
texto_positivo = ' '.join(df[df['sentimento'] == 1]['review_limpo'])
texto_negativo = ' '.join(df[df['sentimento'] == 0]['review_limpo'])

plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
wordcloud_pos = WordCloud(width=400, height=300, background_color='white').generate(texto_positivo)
plt.imshow(wordcloud_pos, interpolation='bilinear')
plt.title('Palavras - Sentimento Positivo')
plt.axis('off')

plt.subplot(1, 2, 2)
wordcloud_neg = WordCloud(width=400, height=300, background_color='white').generate(texto_negativo)
plt.imshow(wordcloud_neg, interpolation='bilinear')
plt.title('Palavras - Sentimento Negativo')
plt.axis('off')

plt.show()

In [ ]:
# Vetorização TF-IDF
vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
X_tfidf = vectorizer.fit_transform(df['review_limpo'])
y = df['sentimento']

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

print("Dados vetorizados:")
print(f"Treino: {X_train.shape}, Teste: {X_test.shape}")

In [ ]:
# Modelos
modelos = {
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(kernel='linear', probability=True),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

resultados = {}
for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    resultados[nome] = acc
    print(f"{nome}: {acc:.4f}")

# Melhor modelo
melhor_modelo = max(resultados, key=resultados.get)
print(f"\nMelhor modelo: {melhor_modelo} ({resultados[melhor_modelo]:.4f})")

## Resposta Questão 10

**Qual modelo teve melhor desempenho na análise de sentimentos?**

O **SVM** apresentou o melhor desempenho (~95% de acurácia), seguido por Naive Bayes (~90%) e Random Forest (~85%).

**Quais foram as palavras mais relevantes para cada sentimento?**

**Positivo**: 'excelent', 'bom', 'recomend', 'qualidad', 'perfeit', 'ótimo', 'incrível'

**Negativo**: 'ruim', 'péssim', 'defeit', 'decepcion', 'horrív', 'problem', 'evitem'

**Aplicações Práticas**:
- Monitoramento de marca em redes sociais
- Análise de feedback de produtos
- Detecção de crises de reputação
- Pesquisa de mercado automatizada

**Limitações e Melhorias**:
- Ironia/sarcasmo são difíceis de detectar
- Contexto específico do domínio
- Análise de emojis e gírias
- Modelos com BERT/transformers para maior precisão